# Parse SEC filings

In [7]:
from __future__ import division, unicode_literals 

import pandas as pd
import numpy as np
import time, sys, os, re
import datetime, pickle, bz2
import getpass
from tqdm import tqdm
import datetime
from datetime import datetime
import string
# Import BeautifulSoup
from bs4 import BeautifulSoup
import re

In [2]:
def cleanStr(test_df_1):
    test_df_1 = test_df_1.translate(str.maketrans('', '', string.punctuation))
    test_df_1 = re.sub('\xa0', ' ', test_df_1)
    test_df_1 = re.sub('\n',' ', test_df_1)
    test_df_1 = re.sub(':','', test_df_1)
    test_df_1 = re.sub("—", '', test_df_1)
    test_df_1 = re.sub('\t', ' ', test_df_1)
    test_df_1 = re.sub(' ·', ' ', test_df_1)
    test_df_1 = re.sub('     ', ' ', test_df_1)
    test_df_1 = re.sub('    ', ' ', test_df_1)
    test_df_1 = re.sub('   ', ' ', test_df_1)
    test_df_1 = re.sub('  ', ' ', test_df_1)
        
    return test_df_1

In [82]:
def cleanDf(test_df):
    test_df.replace('&#160;',' ',regex=True,inplace=True)
    test_df.replace('&nbsp;',' ',regex=True,inplace=True)
    test_df.replace('\.','',regex=True,inplace=True)
    test_df.replace('>','',regex=True,inplace=True)
    test_df.replace('   ',' ',regex=True,inplace=True)
    test_df.replace('  ',' ',regex=True,inplace=True)
    test_df.replace(' ','',regex=True,inplace=True)
    
    return test_df

# unit test

In [3]:
ma_list = sorted(os.listdir("/home/dada/CyberSpace/GQ/AI_prj/10-K/{}".format('Pos')))
ma_list = [i for i in ma_list if ".html" in i]

In [192]:
ma_list[246]

'HIG.html'

In [193]:
with open(f"/home/dada/CyberSpace/GQ/AI_prj/10-K/Pos/{ma_list[246]}") as fp:
    document = BeautifulSoup(fp.read()).get_text().lower()

In [194]:
parse_i_j = BeautifulSoup(document, 'lxml')
parse_i_j = parse_i_j.get_text("\n\n")
parse_i_j = cleanStr(parse_i_j)

In [195]:
parse_i_j[:100]

'inline xbrl viewer this application relies heavily on javascript you will need to allow javascript t'

In [4]:
# # Create the dataframe
# test_df = pd.DataFrame([(x.start(), x.end()) for x in matches])

In [ ]:
regex = re.compile(r'(>item(\s|.| )(7a|7|8)\.{0,1})|(item\s(7a|7|8))', re.IGNORECASE)

In [223]:
matches = regex.finditer(parse_i_j)
test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

test_df.columns = ['item', 'start', 'end']
test_df['item'] = test_df.item.str.lower()

# Display the dataframe
test_df.head()

,item,start,end
0,item 7,256009,256015
1,item 7,256238,256244
2,item 7,258298,258304
3,item 7,267843,267849
4,item 7,275168,275174


In [ ]:
# Get rid of unnesesary charcters from the dataframe
test_df.replace('&#160;',' ',regex=True,inplace=True)
test_df.replace('&nbsp;',' ',regex=True,inplace=True)
test_df.replace('\.','',regex=True,inplace=True)
test_df.replace('>','',regex=True,inplace=True)
test_df.replace('   ',' ',regex=True,inplace=True)
test_df.replace('  ',' ',regex=True,inplace=True)
test_df.replace(' ','',regex=True,inplace=True)
test_df.replace('\.','',regex=True,inplace=True)
test_df.replace('>','',regex=True,inplace=True)

# display the dataframe
test_df.iloc[0, -1]

In [225]:
# Drop duplicates
pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')

# Display the dataframe
pos_dat.reset_index(drop = True, inplace = True)
pos_dat

,item,start,end
0,item7,773904,773910


In [229]:
"item7" in test_df.item.to_list()

True

In [163]:
tt = pd.DataFrame(["item7", 382513, 393456]).T
tt.columns = pos_dat.columns

In [164]:
pos_dat = pd.concat([pos_dat, tt])

In [165]:
pos_dat.reset_index(drop = True, inplace = True)

In [166]:
pos_dat

,item,start,end
0,item7,212496,212502
1,item7a,235286,235293
2,item8,361443,361449
3,item7,382513,393456


In [167]:
pos_dat.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')

,item,start,end
1,item7a,235286,235293
2,item8,361443,361449
3,item7,382513,393456


In [159]:
max(list(pos_dat[pos_dat.item == 'item8'].index+1))

3

In [161]:
pos_dat = pos_dat[:max(list(pos_dat[pos_dat.item == 'item8'].index+1))]
pos_dat

,item,start,end
0,item7,212496,212502
1,item7a,235286,235293
2,item8,361443,361449


In [77]:
# Get Item 7
item_7 = parse_i_j[int(pos_dat.start[0]):int(pos_dat.start[len(pos_dat)-1])]

In [ ]:
item_7 = parse_i_j[int(pos_dat.start[pos_dat.item == 'item7']):int(pos_dat.start[pos_dat.item == 'item7a'])]

In [252]:
with open(f"/home/dada/CyberSpace/GQ/AI_prj/10-K/Pos/ECPG.html") as fp:
    document = BeautifulSoup(fp.read()).get_text().lower()

In [253]:
parse_i_j = BeautifulSoup(document, 'lxml')
parse_i_j = parse_i_j.get_text("\n\n")
parse_i_j = cleanStr(parse_i_j)

In [211]:
matches = re.compile(r'(>item\s(7[\.\s]|7a[\.\s]|8[\.\s])|'
                      'management.s\sdiscussion\sand\sanalysis\sof\s(consolidated\sfinancial|financial)\scondition|'
                      '(consolidated\sfinancial|financial)\sstatements\sand\ssupplementary\sdata)', re.IGNORECASE)

In [212]:
matches_array = pd.DataFrame([(match.group(), match.start(), match.end()) for match in matches.finditer(parse_i_j)])

In [216]:
matches_array.columns = ['item', 'start', 'end']
matches_array

,item,start,end
0,financialstatementsandsupplementarydata,255281,255324
1,management’sdiscussionandanalysisoffinancialco...,267850,267909
2,management’sdiscussionandanalysisoffinancialco...,427034,427093


In [231]:
def prepText_K(ma_status = "Pos"):
    
    ma_dict = {} 
    
    ma_list = sorted(os.listdir("/home/dada/CyberSpace/GQ/AI_prj/10-K/{}".format(ma_status)))
    ma_list = [i for i in ma_list if ".html" in i]
    
    for i in tqdm(ma_list):
        
        ticker_i = i.replace(".html", "")    
        with open(f"/home/dada/CyberSpace/GQ/AI_prj/10-K/{ma_status}/{i}") as fp:
            document = BeautifulSoup(fp.read()).get_text().lower()
            
#            if "item 2" in document:
#             regex_i_j = regex.finditer(document)
#             match_df = pd.DataFrame([(x.start(), x.end()) for x in regex_i_j])
#             parse_i_j = BeautifulSoup(document[match_df.iloc[0,1]:match_df.iloc[len(match_df)-1,0]], 'lxml')

            parse_i_j = BeautifulSoup(document, 'lxml')
            parse_i_j = parse_i_j.get_text("\n\n")
            parse_i_j = cleanStr(parse_i_j)

#             matches_array = pd.DataFrame([(match.group(), match.start()) for match in matches.finditer(parse_i_j)])
#             matches_array.columns = ['SearchTerm', 'Start']
                        
            regex = re.compile(r'(>item(\s| | )(7A|7|8)\.{0,1})|(item\s(7A|7|8))', re.IGNORECASE)
            matches = regex.finditer(parse_i_j)

            test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])
            
            if len(test_df) == 0:
                print(f"{ticker_i} is not processed")
                pass
            
            else:
                test_df.columns = ['item', 'start', 'end']            
                test_df = cleanDf(test_df)
                
                if "item8" in test_df.item.to_list():
                    test_df = test_df[:max(list(test_df[test_df.item == 'item8'].index+1))]
            
                    pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')
                    pos_dat.reset_index(drop = True, inplace = True)
                    ma_dict[ticker_i] = parse_i_j[int(pos_dat.start[0]):int(pos_dat.start[len(pos_dat)-1])]
                else:
                    print(f"{ticker_i} is not processed")
                    pass
            
    return ma_dict

# extract Neg text for training

In [ ]:
%%time 

ma_pos_10K = prepText_K(ma_status = "Pos")

 17%|█▋        | 95/568 [03:18<36:42,  4.66s/it]

C is not processed


 40%|███▉      | 226/568 [07:50<11:43,  2.06s/it]

In [ ]:
ma_pos_10K_df = pd.DataFrame.from_dict(ma_pos_10K, orient ='index')

In [ ]:
print(ma_pos_10K_df.shape)
ma_pos_10K_df.head()

In [ ]:
pickle.dump(ma_pos_10K_df.reset_index(drop=True), open("ma_pos_10K_new.pkl", "wb"))

# extract Neg text for training

In [ ]:
%%time 

ma_neg_10K = prepText_K(ma_status = "Neg")

In [ ]:
ma_neg_10K_df = pd.DataFrame.from_dict(ma_neg_10K, orient ='index')

In [ ]:
print(ma_neg_10K_df.shape)
ma_neg_10K_df.head()

In [ ]:
pickle.dump(ma_neg_10K_df.reset_index(drop=True), open("ma_neg_10K_new.pkl", "wb"))

# extract Neg text for testing

In [ ]:
%%time 

tst_neg_10K = prepText_K(ma_status = "Tst/Neg")

In [ ]:
tst_neg_10K_df = pd.DataFrame.from_dict(tst_neg_10K, orient ='index')

In [ ]:
print(tst_neg_10K_df.shape)
tst_neg_10K_df.head()

In [ ]:
pickle.dump(tst_neg_10K_df.reset_index(drop=True), open("tst_neg_10K_new.pkl", "wb"))

# extract Neg text

In [ ]:
%%time 

tst_pos_10K = prepText_K(ma_status = "Tst/Pos")

In [ ]:
tst_pos_10K_df = pd.DataFrame.from_dict(tst_pos_10K, orient ='index')

In [ ]:
print(tst_pos_10K_df.shape)
tst_pos_10K_df.head()

In [ ]:
pickle.dump(tst_pos_10K_df.reset_index(drop=True), open("tst_pos_10K_new.pkl", "wb"))